In [24]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [25]:
layers = 4
n_timesteps = 60
features = 4
n_batch = 4096
n_epochs = 100

In [26]:
def make_model():
    close_history = Input((n_timesteps, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=4, input_shape=(n_timesteps, 1), return_sequences=True))
    lstm.add(LSTM(units=4, return_sequences=True))
    lstm.add(LSTM(units=4, return_sequences=True))
    lstm.add(LSTM(units=4, return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

In [27]:
model = make_model()

In [28]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 60, 1)]      0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 4)            528         input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 8)            0           sequential_2[0][0]               
                                                                 input_6[0][0]              

In [29]:
model.compile(loss='mse', optimizer=Adam())

In [31]:
model.compile(optimizer=Adam(lr=1e-4), loss='mse')
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/100
1699840/5001241 [=========>....................] - ETA: 5:27 - loss: 146761.3288

KeyboardInterrupt: 

In [2]:
df = pd.read_csv('../options-df-sigma.csv')
df = df.dropna(axis=0)
df = df.drop(columns=['exdate', 'impl_volatility', 'volume', 'open_interest', 'sigma_20'])
df.strike_price = df.strike_price / 1000
call_df = df[df.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = df[df.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [11]:
underlying = pd.read_csv('./daily-closing-prices.csv')

In [12]:
df.tail()

NameError: name 'df' is not defined

In [13]:
underlying.head(20)

,date,close
0,19960102,620.73
1,19960103,621.32
2,19960104,617.70
3,19960105,616.71
4,19960108,618.46
5,19960109,609.45
6,19960110,598.48
7,19960111,602.69
8,19960112,601.81
9,19960115,599.82


In [14]:
padded = np.insert(underlying.close.values, 0, np.array([np.nan] * n_timesteps))

In [15]:
rolled = np.column_stack([np.roll(padded, i) for i in range(n_timesteps)])

In [16]:
rolled = rolled[~np.isnan(rolled).any(axis=1)]

In [17]:
rolled.shape

(5480, 60)

In [18]:
rolled = np.column_stack((underlying.date.values[n_timesteps - 1:], rolled))

In [19]:
price_history = pd.DataFrame(data=rolled)

In [17]:
joined = df.join(price_history.set_index(0), on='date')

In [18]:
call_df = joined[joined.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = joined[joined.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [19]:
call_df = call_df.drop(columns=['date'])
put_df = put_df.drop(columns=['date'])

In [20]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1).values,
                                                                        ((call_df.best_bid + call_df.best_offer) / 2).values,
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1).values,
                                                                    ((put_df.best_bid + put_df.best_offer) / 2).values,
                                                                    test_size=0.01, random_state=42)

In [21]:
call_X_train = [call_X_train[:, -n_timesteps:].reshape(call_X_train.shape[0], n_timesteps, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -n_timesteps:].reshape(call_X_test.shape[0], n_timesteps, 1), call_X_test[:, :4]]
put_X_train = [put_X_train[:, -n_timesteps:].reshape(put_X_train.shape[0], n_timesteps, 1), put_X_train[:, :4]]
put_X_test = [put_X_test[:, -n_timesteps:].reshape(put_X_test.shape[0], n_timesteps, 1), put_X_test[:, :4]]

In [22]:
# model.compile(loss='mse', optimizer=Adam(lr=1e-7))
# history = model.fit(call_X_train, call_y_train, 
#                     batch_size=n_batch, epochs=n_epochs, 
#                     validation_split = 0.01,
#                     callbacks=[TensorBoard()],
#                     verbose=1)

In [20]:
def generate_lstm_model():
    close_history = Input((n_timesteps, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(Bidirectional(LSTM(units=8, input_shape=(n_timesteps, 1), return_sequences=True)))
    lstm.add(Bidirectional(LSTM(units=8, return_sequences=True)))
    lstm.add(Bidirectional(LSTM(units=8, return_sequences=True)))
    lstm.add(Bidirectional(LSTM(units=8, return_sequences=False)))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

In [21]:
lstm_model = generate_lstm_model()

In [22]:
lstm_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 60, 1)]      0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 16)           5440        input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 20)           0           sequential_1[0][0]               
                                                                 input_4[0][0]              

In [23]:
lstm_model.compile(optimizer=Adam(lr=1e-4), loss='mse')
history = lstm_model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

NameError: name 'call_X_train' is not defined